<a href="https://colab.research.google.com/github/smorf-ntsg/example-scripts/blob/master/Earth_Engine_ML_TensorFlow_G4G_2023_model_hosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Earth Engine Neural Networks 2.3: Model Hosting on Vertex AI

Welcome to Geo for Good 2023 Neural Networks 2.3!  Following the model training in Neural Networks 2.2, this notebook demonstrates hosting a model on Vertex AI and connecting to it from Earth Engine.

In [ ]:
from google.colab import auth
import google
import ee
from tensorflow import keras
import tensorflow as tf

In [ ]:
PROJECT = 'YOUR-PROJECT'
BUCKET = 'YOUR-BUCKET'

In [ ]:
auth.authenticate_user()
credentials, _ = google.auth.default()
ee.Initialize(credentials, project=PROJECT, opt_url='https://earthengine-highvolume.googleapis.com')

In [ ]:
REGION = 'us-central1'
TRAINED_MODEL_DIR = 'gs://ee-docs-demos/g4g-2023-nn/trained-model'
MODEL_DIR = f'gs://{BUCKET}/trained-model-eeified'
INPUT_BANDS = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7',
          'B8', 'B8A', 'B9', 'B11', 'B12']
MODEL_NAME = 'g4g-2023-model'
ENDPOINT_NAME = 'g4g-2023-endpoint'
CONTAINER_IMAGE = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest'
ATTRIBUTION = 'Map Data © Google Earth Engine'

In [ ]:
trained_model = keras.models.load_model(TRAINED_MODEL_DIR)

## de/serialization

Vertex AI online prediction expects `base64` encoded inputs and returns `base64` encoded outputs.

In [ ]:
class DeserlializeInput(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def call(self, inputs_dict):
    return {
      k: tf.map_fn(lambda x: tf.io.parse_tensor(x, tf.float32),
                   tf.io.decode_base64(v),
                   fn_output_signature=tf.float32)
        for (k, v) in inputs_dict.items()
    }

  def get_config(self):
    config = super().get_config()
    return config


class ReserlializeOutput(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def call(self, output_tensor, name):
    return {name: tf.identity(tf.map_fn(
        lambda x: tf.io.encode_base64(tf.io.serialize_tensor(x)),
        output_tensor,
        fn_output_signature=tf.string),
                       name=name)}

  def get_config(self):
    config = super().get_config()
    return config


input_deserializer = DeserlializeInput()
output_deserilaizer = ReserlializeOutput()

serlialized_inputs = {
    b: tf.keras.Input(shape=[], dtype='string', name=b) for b in INPUT_BANDS}

updated_model_input = input_deserializer(serlialized_inputs)
updated_model = trained_model(updated_model_input)
updated_model = output_deserilaizer(updated_model, 'probability')
updated_model= tf.keras.Model(serlialized_inputs, updated_model)

In [ ]:
tf.keras.utils.plot_model(updated_model)

## Save the trained model

In [ ]:
updated_model.save(MODEL_DIR)

## Host the trained model

In [ ]:
!gcloud ai models upload \
  --artifact-uri={MODEL_DIR} \
  --project={PROJECT} \
  --region={REGION} \
  --container-image-uri={CONTAINER_IMAGE} \
  --description={MODEL_NAME} \
  --display-name={MODEL_NAME} \
  --model-id={MODEL_NAME}

In [ ]:
!gcloud ai endpoints create \
  --display-name={ENDPOINT_NAME} \
  --region={REGION} \
  --project={PROJECT}

In [ ]:
ENDPOINT_ID = !gcloud ai endpoints list \
  --project={PROJECT} \
  --region={REGION} \
  --filter=displayName:{ENDPOINT_NAME} \
  --format="value(ENDPOINT_ID.scope())"
ENDPOINT_ID = ENDPOINT_ID[-1]
print(ENDPOINT_ID)

!gcloud ai endpoints deploy-model {ENDPOINT_ID} \
  --project={PROJECT} \
  --region={REGION} \
  --model={MODEL_NAME} \
  --display-name={MODEL_NAME}

## Connect to the hosted model from Earth Engine

In [ ]:
print('Prediction link:')
print(f'https://code.earthengine.google.com/690804542fb5e64fb4f04e625a1a363f#project={PROJECT};endpoint={ENDPOINT_ID}foo;')

## Clean up

To avoid incurring costs, delete all buckets, models and endpoints created through this tutorial.